# 03. Azure AI Document Intelligence - Receipt Model

> https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-4.0.0

## A. Create an AI Document Intelligence resource and set up environment to run notebook

**_Prerequsite_:** <br>

**AI Document Intelligence resource**: <br>
To create a AI Document Intelligence resource in your Azure subscription:
Please follow the steps as specified https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0


Get your newly created Document Intelligence service in the Azure portal and on the **Keys and Endpoint** page, copy the **Key1** and **Endpoint** values and paste them in the code cell below, replacing **YOUR_FORM_KEY** and **YOUR_FORM_ENDPOINT**.

**Environment: **

1. **AML workspace**: Please ensure you have Python 3.10 version or above ie select **Python 3.10 - SDK v2** as kernel in AML Notebook. <br>
2. **VS Code**: Please select Python **Python 3.10** above or try to set up virtual envi by following steps https://code.visualstudio.com/docs/python/environments


Please skip this step if you are already done it

## B. Install AI Doc Intelligence library

In [1]:
# install azure-ai-formrecognizer python library and restart the kernal after installation
# ignore this step is you have already installed azure-ai-formrecognizer python library
%pip install azure-ai-formrecognizer --upgrade --user

Note: you may need to restart the kernel to use updated packages.


## C. Setting up AI Doc Intelligence endpoint and key

In [1]:
# Import the os module for interacting with the operating system
import os
# Import for handling resource not found errors
from azure.core.exceptions import ResourceNotFoundError
# Import for authenticating with the Azure service
from azure.core.credentials import AzureKeyCredential
# Import for analysing forms
from azure.ai.formrecognizer import DocumentAnalysisClient, AnalyzeResult

from dotenv import load_dotenv

load_dotenv("credentials.env")
api_key=os.getenv("FORM_RECOGNIZER_KEY")
endpoint = os.getenv("FORM_RECOGNIZER_ENDPOINT") 

form_recognizer_client = DocumentAnalysisClient(endpoint, AzureKeyCredential(api_key))

In [2]:
print(endpoint)

https://eastus.api.cognitive.microsoft.com/


## D. Extract the Receipt Document insights

In [3]:
# Define the URL of the sample receipt document to analyze
# You can change the URL pointing to your sample recepit docs but ensure you provide appropriate access
receiptUrl = "XXXX"

# Start the analysis of the receipt document using the prebuilt receipt model
# The result is a poller object that can be used to check the status of the operation
poller = form_recognizer_client.begin_analyze_document_from_url("prebuilt-receipt",receiptUrl)

# Print the poller object. This is useful for debugging purposes to ensure the analysis has started correctly.
print(poller)

# Get the result of the analysis
result = poller.result()

# Print the documents in the result. This will display the analyzed information from the receipt document.
print(result.documents)

# Loop through each receipt in the result
for idx, receipt in enumerate(result.documents):
    # Print the receipt number and type
    print(f"--------Analysis of receipt #{idx + 1}--------")
    print(f"Receipt type: {receipt.doc_type if receipt.doc_type else 'N/A'}")
    merchant_name = receipt.fields.get("MerchantName")
    if merchant_name:
        print(# Extract and print the merchant name and its confidence level
            f"Merchant Name: {merchant_name.value} has confidence: "
            f"{merchant_name.confidence}"
        )
        # Extract and print the transaction date and its confidence level
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print(
            f"Transaction Date: {transaction_date.value} has confidence: "
            f"{transaction_date.confidence}"
        )
    # If there are items in the receipt, loop through each item and print its details and their confidence levels    
    if receipt.fields.get("Items"):
        print("Receipt items:")
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print(f"...Item #{idx + 1}")
            item_description = item.value.get("Description")
            if item_description:
                print(
                    f"......Item Description: {item_description.value} has confidence: "
                    f"{item_description.confidence}"
                )
            item_quantity = item.value.get("Quantity")
            if item_quantity:
                print(
                    f"......Item Quantity: {item_quantity.value} has confidence: "
                    f"{item_quantity.confidence}"
                )
            item_price = item.value.get("Price")
            if item_price:
                print(
                    f"......Individual Item Price: {item_price.value} has confidence: "
                    f"{item_price.confidence}"
                )
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print(
                    f"......Total Item Price: {item_total_price.value} has confidence: "
                    f"{item_total_price.confidence}"
                )
    # Extract and print the subtotal and its confidence level            
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print(f"Subtotal: {subtotal.value} has confidence: {subtotal.confidence}")
    # Extract and print the total tax and its confidence level
    tax = receipt.fields.get("TotalTax")
    if tax:
        print(f"Total tax: {tax.value} has confidence: {tax.confidence}")
    # Extract and print the tip and its confidence level
    tip = receipt.fields.get("Tip")
    if tip:
        print(f"Tip: {tip.value} has confidence: {tip.confidence}")
    # Extract and print the total and its confidence level
    total = receipt.fields.get("Total")
    if total:
        print(f"Total: {total.value} has confidence: {total.confidence}")


[AnalyzedDocument(doc_type=receipt.retailMeal, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.0, y=0.0), Point(x=1688.0, y=0.0), Point(x=1688.0, y=3000.0), Point(x=0.0, y=3000.0)])], spans=[DocumentSpan(offset=0, length=205)], fields={'Items': DocumentField(value_type=list, value=[DocumentField(value_type=dictionary, value={'Description': DocumentField(value_type=string, value='Cappuccino', content=Cappuccino, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=318.0, y=1583.0), Point(x=655.0, y=1597.0), Point(x=654.0, y=1688.0), Point(x=313.0, y=1678.0)])], spans=[DocumentSpan(offset=103, length=10)], confidence=0.993), 'Quantity': DocumentField(value_type=float, value=1.0, content=1, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=245.0, y=1582.0), Point(x=293.0, y=1583.0), Point(x=288.0, y=1676.0), Point(x=240.0, y=1672.0)])], spans=[DocumentSpan(offset=101, length=1)], confidence=0.995), 'TotalPrice': DocumentField(value_type=float,

## E. Extracted Layout Document insights/ response as a JSON format 

In [4]:
# Import necessary libraries
# json for handling JSON data
# datetime and time for generating timestamps
# AzureJSONEncoder for serializing Python objects to JSON
# urlparse for parsing URLs
import json
import datetime
import time
from azure.core.serialization import AzureJSONEncoder
from urllib.parse import urlparse


# Generate a unique filename based on the current timestamp and the basename of the URL
filename = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S')+"_"+os.path.splitext(os.path.basename(urlparse(receiptUrl).path))[0]

# Convert the result of the analysis (which is a model) to a dictionary
analyze_result_dict = result.to_dict()

# Open a new file with the generated filename and write the dictionary to it as JSON
# Use the AzureJSONEncoder to handle any types that aren't serializable by default
with open(str(filename), 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder,indent=4)

# Convert the dictionary back to the original model
model = AnalyzeResult.from_dict(analyze_result_dict)

# Print some information about the model
print("--------------JSON Response from Model Starts---------------------")

print("Model ID: '{}'".format(model.model_id))
print("Number of pages analyzed {}".format(len(model.pages)))
print("API version used: {}".format(model.api_version))
# Print the dictionary (which is the JSON response from the model)
print(json.dumps(analyze_result_dict,cls=AzureJSONEncoder,indent=4))
print("--------------JSON Response from Model Ends---------------------")

--------------JSON Response from Model Starts---------------------
Model ID: 'prebuilt-receipt'
Number of pages analyzed 1
API version used: 2023-07-31
{
    "api_version": "2023-07-31",
    "model_id": "prebuilt-receipt",
    "content": "Contoso\nContoso\n123 Main Street\nRedmond, WA 98052\n987-654-3210\n6/10/2019 13:59\nSales Associate: Paul\n1 Cappuccino\n$2.20\n1 BACON & EGGS\nSunny-side-up\n$9.5\nSub-Total\n$ 11.70\nTax\n$ 1.17\nTip\n$1.63\nTotal\n$14.50",
    "languages": [],
    "pages": [
        {
            "page_number": 1,
            "angle": 0.1418967992067337,
            "width": 1688.0,
            "height": 3000.0,
            "unit": "pixel",
            "lines": [
                {
                    "content": "Contoso",
                    "polygon": [
                        {
                            "x": 307.0,
                            "y": 570.0
                        },
                        {
                            "x": 519.0,
               